## In-lab exercise and homework: DQN

For any but trivial problems, tabular Q-Learning will fail due to the difficulty of storing the value of every state-action pair and the enormous amount of time it takes for every cell in the table to converge.

The first thing we'd like to do, then, is approximate Q[s,a] with a function q(s,a) using fewer than the 177,147 parameters we used with tabular Q-learning.

As a first attempt, we might convert the input state-action pairs to a binary representation and use a linear output for the Q value. We would use a one-hot representation for each of the 9 game cells in s (3 choices) and the 9 possible actions. This would give us 36 inputs. The neural network would be multi-layer. With two fully connected layers of 10 units each, we'd have 10x37+10x11+11=491 parameters. That's a lot less than tabular Q-learning!

However, take a look at DQN. Their model has a NN to process the input images from the Atari console and has one output for each possible action. With this approach, we would have just 27 inputs and 9 outputs. Two fully connected layers of 10 units each would give us 10x28+10x11+9x11=489 parameters, which is similar in size to the above and has the advantage of only
having to be executed once to get the value of every action. We could go with this tiny network or make it a lot bigger and still beat the Q table's size by a wide margin.

So, the first step will be to replace the Q table with a Q network. Go ahead and write your network class in PyTorch.
Deep Q-Learning

Alright, you have a neural network to replace the Q table, but how to learn its parameters?

Take a look again at Mnih et al. (2015). The method they recommend is experience replay in which they store recent state-action-reward tuples in a buffer and train on random subsamples from the buffer.

This is probably overkill for tic-tac-toe, but let's implement it anyway!

You'll have to decide some things, such as what to do if the agent samples an illegal move. Give a negative reward? Give a 0 reward? Think about it and try an approach.

Go ahead and write the DQN algorithm, using your simple fully connected network in place of the CNN. Get it learning!

Report on your experiments and results by next week.

In [1]:
import gym
import tensorflow as tf
import numpy as np
import math
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class DQN(nn.Module):

    def __init__(self, inputs, outputs):
        super(DQN, self).__init__()

        self.fc1 = nn.Linear(inputs, 10)
        self.fc2 = nn.Linear(10, outputs)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## Experience replay

To improve the performance of Q-learning, we use **experience replay**.

**Experience replay** means we store the agent's experiences during an episode instead of running Q-learning. The learning phase with experience replay becomes two phases: gaining experience and updating models based on the experience obtained after an episode finishes. Specifically, the experience (also called the buffer, or memory) includes the past state, the action taken, the reward received, and the next state for individual steps in an episode.

In the learning phase, a certain number of data points are randomly sampled from the experience and are used to train the learning models. Experience replay can stabilize training by providing a set of samples with low correlation, which, as a result, increases learning efficiency.

In [3]:
class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = [] # Queue-like
        self.position = 0

    def push(self, experience):
        """Saves a experience."""
        if len(self.memory) < self.capacity:
            self.memory.append(None) # if we haven't reached full capacity, we append a new transition
        self.memory[self.position] = experience  
        self.position = (self.position + 1) % self.capacity # e.g if the capacity is 100, and our position is now 101, we don't append to
        # position 101 (impossible), but to position 1 (its remainder), overwriting old data

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size) 

    def __len__(self): 
        return len(self.memory)

### DQN algorithm

In [4]:
# Action selection , if stop training == True, only exploitation
def select_action(state):
    best_action = policy_net(state).max(0)[1].item()
    if best_action in available_action
        with torch.no_grad():
            return policy_net(state).max(0)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

# Training 
def optimize_model(state, is_player):
    if len(memory) < BATCH_SIZE:
        print(f'Save experiences to memory. Current memory:{len(memory)}')
        return
    
    if is_player:
        next_state, reward, done = memory.sample(BATCH_SIZE)
        next_state = torch.FloatTensor(next_state).view(27).to(device)
    
    state_batch = torch.cat(state).to(device)
    next_state_batch = torch.cat(next_state).to(device)
    reward_batch = torch.cat(reward).type(torch.FloatTensor).to(device)

    q_value = policy_net(state_batch)
    next_q_value = policy_net(next_state_batch)

    # Compute Q values
    expected_next_q_value = (next_q_value * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(q_value, expected_next_q_value.unsqueeze(1))
    plt.figure(2)

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()



SyntaxError: invalid syntax (<ipython-input-4-9050ed122cf8>, line 4)

In [22]:
import importlib
from collections import defaultdict

game_name = 'tictactoe'
game_module = importlib.import_module("games." + game_name)
env = game_module.Game()

state = env.reset()
available_action = env.legal_actions()

BATCH_SIZE = 5 # original = 128
GAMMA = 0.999 # original = 0.999
N_EPISODES = 50000 # total episodes to be run
MEMORY_SIZE = 100000 # original = 10000
EPS_START = 0.9 # original = 0.9
EPS_END = 0.01 # original = 0.05
EPS_DECAY = 3000 # original = 200


policy_net = DQN(state.flatten().shape[0], len(available_action)).to(device)
optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(MEMORY_SIZE)

player = 1

for episode in range(N_EPISODES):
    if episode % 10000 == 9999:
        print("episode: ", episode + 1)
    state = env.reset()
    state = torch.FloatTensor(state).view(27).to(device)

    is_done = False
    while not is_done:
        if env.to_play() == player:
            available_action = env.legal_actions()
            action = select_action(state)
            next_state, reward, is_done = env.step(action)
            next_state = torch.FloatTensor(next_state).view(27).to(device)
            optimize_model(state, True)
        else:
            action = env.expert_agent()
            next_state, reward, is_done = env.step(action)
            next_state = torch.FloatTensor(next_state).view(27).to(device)

            if is_done:
                reward = -reward
                optimize_model(state, False)
        
        if is_done:
            break

        memory.push((state, reward, is_done))
        state = next_state


print('Complete')

5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8] 1 False
[0, 2, 3, 5, 6, 7, 8

KeyboardInterrupt: 

I cannot finish this on time. I just got the concept but still need to implimented.